# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

Classification. This is classification because the output we want is takes class labels of "needs early intervention" and "doesn't need early intervention". Regression has output that has continuous values like a range of real numbers.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [22]:
# Import libraries
import numpy as np
import pandas as pd

In [23]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print("Student data read successfully!")
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [24]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call

n_students = len(student_data.index) 
n_features = len(student_data.iloc[0]) - 1 #last column is output not feature
n_passed = student_data['passed'].value_counts()['yes']
n_failed = student_data['passed'].value_counts()['no']
grad_rate = (n_passed / (n_failed + n_passed) ) * 100
print("Total number of students: {}".format(n_students))
print("Number of students who passed: {}".format(n_passed))
print("Number of students who failed: {}".format(n_failed))
print("Number of features: {}".format(n_features))
print("Graduation rate of the class: {:.2f}%".format(grad_rate))

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 0.00%


The first thing I notice from this data is that it's a relatively small size. Also ~33% of the students failed so that is the minority group. Since the importance is on the minority class, using an F<sub>1</sub> score might be more appropriate than a simple accuracy measure.

## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [25]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print("Feature column(s):-\n{}".format(feature_cols))
print("Target column: {}".format(target_col))

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print("\nFeature values:-")
print(X_all.head()) # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [26]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems(): 
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print("Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [27]:
from sklearn.cross_validation import train_test_split

# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=.2405, random_state=10)

print("Training set: {} samples".format(X_train.shape[0]))
print("Test set: {} samples".format(X_test.shape[0]))
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

1 - Decision Tree. A decision tree is a machine learning model that can be used for supervised classification. Some of its strengths are that it's easy to conceptualize the data with it, it allows simple decision rules in each node, and it is easy to explain the model to decision makers. Some weaknesses are that it is prone to overfitting, grows exponentially in size with each level of the tree, can be sensitive to small changes in data, and it can have problems with out-of-sample predictions.

I chose this as one of the 3 models because it would be easy to explain to the decision makers. My job is meaningless if I cannot convert what information I've learned into some action. Also the type of features and the range of values in this data allow easy rules for splitting the data at the nodes. Also there doesn't seem to exist a data point that could be an extreme outlier based on the possible values for each feature.

2 - Support Vector Machine. A support vector machine is a learning model that can be used for supervised classification. Some of its strengths are that is works well in complicated domains with large margin levels, it's effective in high dimensional spaces, and the kernel functions can allow domain knowledge to be built into the algorithm. Some weaknesses are that the cubic training time makes this not work well with large datasets, it doesn't work well with very noisy data, doesn't inherently provide probability estimates.

I chose this as one of 3 models because our training dataset is not that large and we do not require a probability estimate. Also we have a large number of features in our data this might be a good choice.

3 - K-NN Classifier. A K-nearest neighbors classifier is a machine learning model that can be used for supervised classification. It is an instance based model, that stores instances of the data and does not try to make a generalized model. Some of its strengths are its training run time, it's robust to noisy training data, and is effective with large data sets. Some weakness are that it has a high computation cost since it doesn't form a predictive model with the data initially, and the K value and distance formula can make a big difference in its predictive ability.

I chose this as of the 3 models because we I think we have enough data for it to yield good results. Also even though the computation time might be higher than the other models, the time it actually takes would not make a negative impact on choosing it.  

In [49]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print("Training {}...".format(clf.__class__.__name__))
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print("Done!\nTraining time (secs): {:.8f}".format(end - start))

# TODO: Choose a model, import it and instantiate an object
from sklearn import tree
clf = tree.DecisionTreeClassifier()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print(clf)  # you can inspect the learned model by printing it


Training DecisionTreeClassifier...
Done!
Training time (secs): 0.00499988


In [50]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print("Predicting labels using {}...".format(clf.__class__.__name__))
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print("Done!\nPrediction time (secs): {:.12f}".format(end - start))
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print("F1 score for training set: {}".format(train_f1_score))

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001000165939
F1 score for training set: 1.0


In [51]:
# Predict on test data
print("F1 score for test set: {}".format(predict_labels(clf, X_test, y_test)))

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000999927521
F1 score for test set: 0.698412698413


In [52]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print("------------------------------------------")
    print("Training set size: {}".format(len(X_train)))
    train_classifier(clf, X_train, y_train)
    print("F1 score for training set: {}".format(predict_labels(clf, X_train, y_train)))
    print("F1 score for test set: {}".format(predict_labels(clf, X_test, y_test)))
    return 
    
    
# TODO: Run the helper function above for desired subsets of training data
for training_set_size in [100,200,300]:
    train_predict(clf, X_train[0:training_set_size], y_train[0:training_set_size], X_test, y_test)   
# Note: Keep the test set constant

------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.00199986
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000000000000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001000165939
F1 score for test set: 0.617886178862
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.00300002
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000000000000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000000000000
F1 score for test set: 0.681818181818
------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.00399995
Predicting labels using DecisionTreeClassifier...
Done!
Predic

In [53]:
from IPython.display import display
#columns = [['Training Set Size'],[100,200,300]]
#df = pd.DataFrame({100:[0.00100017,0.000000000000,0.8085106382978724,0.7464788732394365],\
#                   200:[1,2,3, 4],300:[1,2,3, 4]}, \
#                  index=['Training time (secs)', 'Prediction time (secs)', 'F1 score for training set', 
  #                       \'F1 score for test set'])
df = pd.DataFrame({100:[0.00199986,0.000000000000,1.0,0.617886178862], 
                   200:[0.00300002,0.000000000000,1.0,0.681818181818],     
                   300:[0.00599980,0.001000165939,1.0,0.688]},                        
                  index=['Training time (secs)', 'Training Data Prediction time (secs)', 'F1 score for training set','F1 score for test set'])
print('Stats for Decision Tree Classifier')
display(df)



Stats for Decision Tree Classifier


,100,200,300
Training time (secs),0.002000,0.003000,0.006
Training Data Prediction time (secs),0.000000,0.000000,0.001
F1 score for training set,1.000000,1.000000,1.000
F1 score for test set,0.617886,0.681818,0.688


In [33]:
# Train and predict using SVM
from sklearn import svm
clf = svm.SVC()

# Fit model to training data
train_classifier(clf, X_train, y_train)

for training_set_size in [100,200,300]:
    train_predict(clf, X_train[0:training_set_size], y_train[0:training_set_size], X_test, y_test)  


Training SVC...
Done!
Training time (secs): 0.02300000
------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.00200009
Predicting labels using SVC...
Done!
Prediction time (secs): 0.000999927521
F1 score for training set: 0.83660130719
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001999855042
F1 score for test set: 0.822784810127
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.00600004
Predicting labels using SVC...
Done!
Prediction time (secs): 0.003000020981
F1 score for training set: 0.855218855219
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002000093460
F1 score for test set: 0.794701986755
------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.00899982
Predicting labels using SVC...
Done!
Prediction time (secs): 0.006000041962
F1 score for training set: 0.861471861472
P

In [54]:
df = pd.DataFrame({100:[0.00200009,0.000999927521,0.83660130719,0.822784810127], 
                   200:[0.00600004,0.003000020981,0.855218855219,0.794701986755],     
                   300:[0.00899982,0.006000041962,0.861471861472,0.807947019868]},                        
                  index=['Training time (secs)', 'Training Data Prediction time (secs)', 'F1 score for training set','F1 score for test set'])
print('Stats for Support Vector Machine')
display(df)

Stats for Support Vector Machine


,100,200,300
Training time (secs),0.002000,0.006000,0.009000
Training Data Prediction time (secs),0.001000,0.003000,0.006000
F1 score for training set,0.836601,0.855219,0.861472
F1 score for test set,0.822785,0.794702,0.807947


In [35]:
# Train and predict using K-NN Classifier
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()

# Fit model to training data
train_classifier(clf, X_train, y_train)

for training_set_size in [100,200,300]:
    train_predict(clf, X_train[0:training_set_size], y_train[0:training_set_size], X_test, y_test)  


Training KNeighborsClassifier...
Done!
Training time (secs): 0.00400019
------------------------------------------
Training set size: 100
Training KNeighborsClassifier...
Done!
Training time (secs): 0.00199986
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.004000186920
F1 score for training set: 0.810810810811
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.003999948502
F1 score for test set: 0.776315789474
------------------------------------------
Training set size: 200
Training KNeighborsClassifier...
Done!
Training time (secs): 0.00099993
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.003999948502
F1 score for training set: 0.833333333333
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.003000020981
F1 score for test set: 0.779411764706
------------------------------------------
Training set size: 300
Training KNeighborsClassifier...
Done!
Training time (

In [55]:
df = pd.DataFrame({100:[0.00199986,0.004000186920,0.810810810811,0.776315789474], 
                   200:[0.00099993,0.003999948502,0.833333333333,0.779411764706],     
                   300:[0.00099993,0.009999990463,0.842105263158,0.771428571429]},                        
                  index=['Training time (secs)', 'Training Data Prediction time (secs)', 'F1 score for training set','F1 score for test set'])
print('Stats for K-NN Classifier')
display(df)

Stats for K-NN Classifier


,100,200,300
Training time (secs),0.002000,0.001000,0.001000
Training Data Prediction time (secs),0.004000,0.004000,0.010000
F1 score for training set,0.810811,0.833333,0.842105
F1 score for test set,0.776316,0.779412,0.771429


## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

Based on my experiments I chose the support vector machine as the best model. This model had the higher F<sub>1</sub> score for all subsets of data points. 

F<sub>1</sub> score is a measurement of a system's precision and recall ability. The F<sub>1</sub> score ranges from 0 - 1 where 1 is the best and perfectly predicts the data. Precision is a measure of how often the model predicts a label correctly compared with the total number of times it predicted that label. For instance if this model correctly identified 8 students that needed intervention and predicted 2 other students that it thought needed intervention but really didn't, then it would have a precision of .8 . Recall is the amount of times a model predicts a label correctly compared with the total number actual times that label was true. For instance if this model correctly identified 8 students that needed intervention and there were 8 other students that needed intervention that it predicted didn't need intervention, then it would have a recall of .5 .  

In practical terms for our use, precision can be thought of as the ratio of students that we helped to the total students that we helped. Recall can be thought of as the ratio of students that we helped to the total students that needed help. If I had to choose which is more important out of the two, I would prefer a higher recall. This is because we can help more of those students that need help. To achieve a perfect recall we could intervene for each student, but this method does not seem cost effective.  

Besides just looking at F<sub>1</sub> score, other factors to consider are the amount of data, performance, and resource costs the different models will take. All 3 of these models were appropriate based on the data, cost, and performance so the leading factor was just the best F<sub>1</sub> score. The training and prediction times for all 3 models were negligible.

To understand SVM let's pretend you are a parent with twins who are about to go to separate college. They share a room and have boxed up everything that they will bring with them. They have asked you to pack those boxes into their respective vehicles and you kindly agreed. As you enter the room you find 40 boxes scattered about. 30 of these boxes have the name of either one of your twins on them, but the remaining 10 do not. SVM treats these 30 boxes as the training data and it will find a way to divide the room so that there exists the most distance between this line and either twin's side of boxes. Since you are a smart person, you notice this division too and you imagine this line in your head. Now these 10 unlabeled boxes are like the testing data from our model. Since you can see what side of the room they are on relative to that division you made earlier, you can predict which twin it belongs to. To see how well our model works, you can open up these boxes and inspect the contents to see how well it did.  

In our chosen untuned SVM model we had a F<sub>1</sub> score of 0.807947 .


In [90]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn import grid_search
from sklearn.metrics import make_scorer

scorer = make_scorer(f1_score, pos_label='yes')   
parameters = {'C':[x for x in [.1,.2,.5,.7,1,2]], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}

clf = svm.SVC()
clf = grid_search.GridSearchCV(clf, parameters, scoring=scorer)
train_predict(clf, X_train, y_train, X_test, y_test)  


------------------------------------------
Training set size: 300
Training GridSearchCV...
Done!
Training time (secs): 0.83099985
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.005000114441
F1 score for training set: 0.861471861472
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.000999927521
F1 score for test set: 0.807947019868


The F<sub>1</sub> score of 0.807947019868 is the same as before I used grid_search to tune it. This shows the default settings of the SVM had the best results. My impression is that if we had more data our model would provide a better score with tuning.